In [2]:
import os
os.getcwd()

'D:\\Yifeng -- Project Work\\ys8mz_sandbox\\degree_completion_4'

In [3]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import confusion_matrix

In [4]:
df = pd.read_csv("pred_score_by_race.csv")

In [5]:
def create_confusion_matrix_2(y_test, y_test_pred, threshold):
    cm_arr = confusion_matrix(y_test, np.where(y_test_pred > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_0','Pred_1'], index=['Real_0', 'Real_1'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    return p1,r1,p0,r0,round(2*p1*r1/(p1+r1),4)

In [6]:
def two_side_z_test(n1, p1, n2, p2):
    z = (p2-p1)/np.sqrt(p2*(1-p2)/(n2-1)+p1*(1-p1)/(n1-1))
    return 2*(1-stats.norm.cdf(np.abs(z)))

In [7]:
new_pred_real = pd.DataFrame({'pred_y': list(df.pred_y),'real_y': list(df.test_y), 'race': list(df.race_column)})

In [8]:
white_df = new_pred_real[new_pred_real.race == "white"]
white_df = white_df.iloc[np.argsort(white_df.pred_y), :]
afam_df = new_pred_real[new_pred_real.race == "afam"]
afam_df = afam_df.iloc[np.argsort(afam_df.pred_y), :]

In [9]:
results = []
for qq in [0.1, 0.2, 0.25, 0.3, 0.33, 0.4, 0.5, 0.6, 0.67, 0.7, 0.75, 0.8, 0.9]:
    print("qq = {}".format(qq))
    q = np.quantile(new_pred_real.pred_y, qq)
    new_pred_real_pct = new_pred_real[new_pred_real.pred_y < q]
    new_pred_real_pct = new_pred_real_pct[new_pred_real_pct.race.apply(lambda x: x in {'white', 'afam'})]
    new_pred_real_pct_agg = new_pred_real_pct.groupby(['race']).agg({'real_y': 'mean', 'pred_y': 'count'}).sort_index().loc[:,['real_y', 'pred_y']]
    n1, p1 = new_pred_real_pct_agg.iloc[0,1], new_pred_real_pct_agg.iloc[0,0]
    n2, p2 = new_pred_real_pct_agg.iloc[1,1], new_pred_real_pct_agg.iloc[1,0]
    p_value_1 = two_side_z_test(n1, p1, n2, p2)
    ratio_1 = n1 / n2
    pp1, pp2 = 0, 1
    j = 1
    # while p_value < 0.05:
    while pp1 < pp2 and j < n2 and n1 + j <= afam_df.shape[0]:
        j += 1
        white_df_new = white_df.iloc[0:(n2-j),:]
        afam_df_new = afam_df.iloc[0:(n1+j),:]
        merged_new = pd.concat([white_df_new, afam_df_new])
        merged_new_agg = merged_new.groupby(['race']).agg({'real_y': 'mean', 'pred_y': 'count'}).sort_index().loc[:,['real_y', 'pred_y']]
        nn1, pp1 = merged_new_agg.iloc[0,1], merged_new_agg.iloc[0,0]
        nn2, pp2 = merged_new_agg.iloc[1,1], merged_new_agg.iloc[1,0]
    p_value_2 = two_side_z_test(nn1, pp1, nn2, pp2)
    ratio_2 = nn1/nn2
    results.append((int(qq*100), p1, p2, n1, n2, pp1, pp2, nn1, nn2, nn1 - n1))

qq = 0.1
qq = 0.2
qq = 0.25
qq = 0.3
qq = 0.33
qq = 0.4
qq = 0.5
qq = 0.6
qq = 0.67
qq = 0.7
qq = 0.75
qq = 0.8
qq = 0.9


In [10]:
results_df = \
pd.DataFrame(results, columns=["bottom_%", "success_rate_black", "success_rate_white", "n1", "n2",
                               "success_rate_black", "success_rate_white", "nn1","nn2",
                               "black_increased"]).sort_values(["bottom_%"]).round(4)
results_df.to_csv("calibration_bias.csv", index=False)

In [11]:
white_df.shape[0], afam_df.shape[0]

(32288, 16796)

In [12]:
def std_new(p,n):
    return np.sqrt(p*(1-p)/n)

In [13]:
results = []
for qq in [0.1, 0.2, 0.25, 0.3, 0.33, 0.4, 0.5, 0.6, 0.67, 0.7, 0.75, 0.8, 0.9]:
    print("qq = {}".format(qq))
    q = np.quantile(new_pred_real.pred_y, qq)
    new_pred_real_pct = new_pred_real[new_pred_real.pred_y < q]
    new_pred_real_pct = new_pred_real_pct[new_pred_real_pct.race.apply(lambda x: x in {'white', 'afam'})]
    new_pred_real_pct_agg = new_pred_real_pct.groupby(['race']).agg({'real_y': 'mean', 'pred_y': 'count'}).sort_index().loc[:,['real_y', 'pred_y']]
    n1, p1 = new_pred_real_pct_agg.iloc[0,1], 1 - new_pred_real_pct_agg.iloc[0,0]
    n2, p2 = new_pred_real_pct_agg.iloc[1,1], 1 - new_pred_real_pct_agg.iloc[1,0]
    std1 = std_new(p1, n1)
    std2 = std_new(p2, n2)
    white = new_pred_real[np.array(new_pred_real.race == "white") & np.array(new_pred_real.real_y == 0)]
    afam = new_pred_real[np.array(new_pred_real.race == "afam") & np.array(new_pred_real.real_y == 0)]
    n4 = white.shape[0]
    n3 = afam.shape[0]
    p4 = np.mean(white.pred_y < q)
    p3 = np.mean(afam.pred_y < q)
    std3 = std_new(p3, n3)
    std4 = std_new(p4, n4)
    results.append((qq, p1, std1, p2, std2, p3, std3, p4, std4))

qq = 0.1
qq = 0.2
qq = 0.25
qq = 0.3
qq = 0.33
qq = 0.4
qq = 0.5
qq = 0.6
qq = 0.67
qq = 0.7
qq = 0.75
qq = 0.8
qq = 0.9


In [15]:
results_df = \
pd.DataFrame(results, columns=["bottom_%", 
                               "black", "std_black",
                               "white", "std_white",
                               "black", "std_black",
                               "white", "std_white"]).sort_values(["bottom_%"]).round(4)
results_df.to_csv("precision0_tnr_bias.csv", index=False)